# AB testing

Представим, что мы работаем аналитиком в онлайн-кинотеатре. \
Мы хотим добавить новую красивую кнопку "купить подписку" на страницу, где ее раньше не было. \
Предполагается, что добавив эту кнопку, мы увеличим конверсию в покупку подписки. До проведения эксперимента значение конверсии равно 12%, и команда считает, что стоит выкатывать это обновление на всех пользователей, если мы увидим увеличение конверсии на 1%. Чтобы проверить, действительно ли эта кнопка увеличит конверсию, мы решили провести АБ-тест.


## Дизайн эксперимента

**1. Формулируем гипотезу:**

        Н0: CRa = CRb 
        H1: CRa != CRb 
        
  CR - conversion rate

**2. Рассчитываем необходимый размер выборки**

Количество необходимых наблюдений зависит от нескольких факторов:
    
- Мощность теста (1-β). Мощность теста представляет собой вероятность отклонить нулевую гипотезу, если она неверна. В нашем эксперименте мощность будет равна 0.8
- Значение альфа (α). Обозначим ее равной 0.05
- Размер эффекта (Effect size). Минимальное значение различия между группами, которое мы хотим обнаружить нашим стат.тестом. В нашем случае мы хотим зафиксировать разницу минимум в 1%
- 

In [64]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns 
from math import ceil 

In [110]:
effect_size = sms.proportion_effectsize(0.12, 0.13)

required_n = sms.NormalIndPower().solve_power(
                                effect_size,
                                power=0.8,
                                alpha=0.05,
                                ratio=1)               

required_n = ceil(required_n)                          

print(required_n)

17164


Таким образом, мы получили, что нам необходимо как минимум **17164 наблюдений** в каждой группе.

## Сбор и обработка данных

В датасете, который используется для проекта уже накоплено около 300 тыс. наблюдений, но для симуляции ситуации, что мы собираем наблюдения возьмем по 17164 наблюдений из каждой группы.

In [111]:
ab = pd.read_csv(r'C:\Users\Honor\Downloads\ab_data.csv')

In [112]:
ab.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [113]:
ab.shape

(294478, 5)

In [114]:
ab.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

Посмотрим, есть ли в датасете пользователей, которые встречаются больше одного раза

In [115]:
session_counts = ab['user_id'].value_counts()
session_counts.count()

290584

In [116]:
multi_users = session_counts[session_counts > 1].count()

print(f"{multi_users} пользователей встречаются в датасете больше одного раза")

3894 пользователей встречаются в датасете больше одного раза


Т.к. количество таких пользователей довольно низкое по сравнению с общим кол-вом пользователей в датасете, мы можем удалить повторяющихся пользователей

In [117]:
users_to_drop = session_counts[session_counts > 1].index

ab = ab[-ab['user_id'].isin(users_to_drop)]

In [118]:
control_sample = ab.query('group == "control"').sample(n=required_n)
treatment_sample = ab.query('group == "treatment"').sample(n=required_n)

In [119]:
print(control_sample.shape)
print(treatment_sample.shape)

(17164, 5)
(17164, 5)


In [120]:
ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)
ab_test.head()

,user_id,timestamp,group,landing_page,converted
0,851335,2017-01-23 02:03:39.820710,control,old_page,1
1,910066,2017-01-06 07:09:04.245559,control,old_page,0
2,910462,2017-01-07 03:08:31.962720,control,old_page,0
3,845518,2017-01-19 23:17:38.578794,control,old_page,0
4,648143,2017-01-20 16:31:03.613631,control,old_page,0


In [121]:
ab_test.user_id.nunique()

34328

In [122]:
conversion_rates = ab_test.groupby('group')['converted']

conversion_rates = conversion_rates.agg(np.mean)
conversion_rates.columns = ['conversion_rate']
conversion_rates

group
control      0.120718
treatment    0.117280
Name: converted, dtype: float64

В контрольной группе значение конверсии оказалось равным **12.1%** \
В тестовой группе **11.8%** 

Значение в контрольной группе выше. Рассмотрим являются ли эти различия стат. значимыми.

## Проверка гипотезы

Для сравнения двух значений конверсий будем использовать z-test. Согласно центральной предельной теореме, мы можем безопасно аппроксимировать распределение биномиальной пропорциональной статистики, используя нормальное распределение.

In [123]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [124]:
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']

In [125]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]


z_stat, pval = proportions_ztest(successes, nobs=nobs)

(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 0.98
p-value: 0.325
ci 95% for control group: [0.116, 0.126]
ci 95% for treatment group: [0.112, 0.122]


Значение p-value оказалось равным 0.325, поэтому мы не имеем оснований отклонить нулевую гипотезу. Это означает, что мы не можем говорить о том, что добавление новой кнопки стат. значимо влияет на конверсию в покупку подписки.